In [ ]:
# #模型下载
# from modelscope import snapshot_download
# # model_dir = snapshot_download('Qwen/Qwen2.5-1.5B')
# model_dir = snapshot_download('deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B')

In [1]:
# import pandas as pd
# import numpy as np

# df = pd.read_csv("train_zh.csv", encoding='utf-8')
# df_split = np.array_split(df, 3)  # 拆分成 3 份

# # 保存拆分后的文件（可选）
# for i, split_df in enumerate(df_split):
#     split_df.to_csv(f"train_zh_part_{i+1}.csv", index=False, encoding='utf-8')
# # train_zh_part_1.csv (4045 rows)
# # train_zh_part_2.csv (4045 rows)
# # train_zh_part_3.csv (4045 rows)

/usr/local/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [4]:
# import pandas as pd
# import numpy as np
# import csv
# # df=pd.read_csv("train_zh_part_1.csv", encoding='utf-8')
# # df
# max_length = 0
# with open('train_zh_part_1.csv', 'r', encoding='utf-8') as f:
#     reader = csv.reader(f)
#     for row in reader:
#         # 将当前行的所有字段拼接为字符串
#         line = ''.join(row)
#         current_length = len(line)
#         if current_length > max_length:
#             max_length = current_length
# print(f"最长的行数据字数为：{max_length}")

# max_length = 0
# with open('train_zh_part_2.csv', 'r', encoding='utf-8') as f:
#     reader = csv.reader(f)
#     for row in reader:
#         # 将当前行的所有字段拼接为字符串
#         line = ''.join(row)
#         current_length = len(line)
#         if current_length > max_length:
#             max_length = current_length
# print(f"最长的行数据字数为：{max_length}")

# max_length = 0
# with open('train_zh_part_3.csv', 'r', encoding='utf-8') as f:
#     reader = csv.reader(f)
#     for row in reader:
#         # 将当前行的所有字段拼接为字符串
#         line = ''.join(row)
#         current_length = len(line)
#         if current_length > max_length:
#             max_length = current_length
# print(f"最长的行数据字数为：{max_length}")

最长的行数据字数为：5595
最长的行数据字数为：7482
最长的行数据字数为：5385


In [1]:
# !python -m pip install --upgrade pip
# !pip config set global.index-url https://pypi.tuna.tsinghua.edu.cn/simple
# !pip install swanlab -i https://pypi.org/simple  # 使用官方PyPI源
# !pip install swanlab -i https://repo.huaweicloud.com/repository/pypi/simple/
from swanlab.integration.transformers import SwanLabCallback

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# config(need to run before training)

In [2]:
# -*- coding: utf-8 -*-
import torch
from transformers import DataCollatorForLanguageModeling
from datasets import Dataset, load_dataset  # 正确的小写导入
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import pandas as pd

# 基础配置
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
# model_name = "Qwen/Qwen2.5-1.5B" #参数量和 gpt2 一样.
# basic_model_path = "./lora_basic_with_all_loss"     # 基础微调结果
basic_model_path = "./lora_basic_deepseek_r1"     # 基础微调结果
sft_model_path = "./lora_sft"        # SFT微调结果
# merged_model_path = "./lora_basic_sft"  # 合并模型
device_map = "auto"


# 加载模型和分词器
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

# moda local model
model_name = "/mnt/workspace/.cache/modelscope/models/Qwen/Qwen2.5-1.5B"  # 替换为实际的本地模型路径

def tokenize_function(examples):
    # 假设 response 以 "<Response>" 开始
    response_start_token_id = tokenizer.encode("<Response>", add_special_tokens=False)[0]
    tokenized = tokenizer(
        examples["text"],
        padding="longest",  # 动态填充到批次中最长样本的长度
        # truncation=True,
        # max_length=512,
        truncation=False,   # 禁用截断
        return_tensors="pt"
    )
    # labels = tokenized["input_ids"].clone()
    # for i in range(len(labels)):
    #     # 找到 <Response> 的起始位置
    #     start_pos = (tokenized["input_ids"][i] == response_start_token_id).nonzero(as_tuple=True)[0]
    #     if len(start_pos) > 0:
    #         labels[i, :start_pos.item()] = -100  # 忽略 instruction + prompt 的 loss
    # tokenized["labels"] = labels
    
    tokenized["labels"] = tokenized["input_ids"].clone()  # 直接复制输入作为标签
    return tokenized

lora_config = LoraConfig(
    r=8,  # 增大 LoRA 矩阵秩 -------------------> next time to 8 #降低秩值
    lora_alpha=32,  # 调整 alpha 值 ------------------> next time to 32 #保持alpha/r=4的比例
    # target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],,  # 扩展目标模块
    target_modules = ["c_attn", "mlp.down_proj", "mlp.up_proj"],  # qwen, 增强特征提取能力
    lora_dropout=0.2,  # 增加 dropout 防止过拟合 ---------------> next time to 0.2 #增加Dropout
    bias="none",
    task_type="CAUSAL_LM",
    # inference_mode=False,  # 确保处于训练模式[5](@ref)
    # modules_to_save=["lm_head"]  # 允许后续微调时更新头部[10](@ref)
)


[2025-05-04 18:53:28,244] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


# basic knowledge(752 rows)

In [25]:
# ==================== 基础知识微调部分 ====================
# 加载基础知识数据集
def load_basic_knowledge_dataset(csv_path):
    df = pd.read_csv(csv_path)
    # 确保只有prompt和response列
    df = df[['prompt', 'response']]
    return Dataset.from_pandas(df)

# 基础知识数据预处理函数
def format_basic_data(example):
    prompt = example["prompt"]
    response = example["response"]
    full_prompt = f"<Prompt>{prompt}</Prompt>\n<Response>{response}</Response>"
    return {"text": full_prompt.replace("\n", "")}

# 加载基础知识数据集
# basic_dataset = load_basic_knowledge_dataset("game_strategy_and_term.csv")
basic_dataset = load_dataset("csv", data_files="game_strategy_and_term.csv")["train"]
# print(basic_dataset["train"])
# basic_dataset = basic_dataset.select(range(50))  # 选择前5个样本
basic_dataset = basic_dataset.map(format_basic_data, remove_columns=["prompt", "response"])
basic_dataset = basic_dataset.train_test_split(test_size=0.1)

# # # 加载前10行数据
# # def load_mini_dataset(csv_path, n_rows=2):
# #     df = pd.read_csv(csv_path, nrows=n_rows)
# #     # print(df)
# #     return Dataset.from_pandas(df)

# # 加载迷你数据集
# # dataset = load_mini_dataset("train_zh.csv")
# # dataset = dataset.map(format_data, remove_columns=["instruction", "prompt", "response", "meta"])
# # # 加载数据集
# dataset = load_dataset("csv", data_files="train_zh.csv")["train"]
# dataset = dataset.select(range(5))  # 选择前5个样本 [[7]] for test.
# dataset = dataset.map(format_data, remove_columns=["instruction", "prompt", "response", "meta"])


In [19]:
basic_dataset['train']['text'][0]

'<Prompt>如何通过悍跳提升狼人杀游戏水平？</Prompt><Response>狼人杀是一款富有策略和心理战的游戏，想要提升水平，悍跳20盘是一个极具挑战性且有效的方法。以下是悍跳对提升狼人杀水平的积极影响及相关建议。1、抓狼心态的提升悍跳过程中，你扮演狼队的领导，需要应对巨大的压力。这锻炼了你在成为狼时的心态和发言状态，使其更接近一位好人。通过经历1打11的情境，你能够提升自己的抓狼信心，因为完美的发言能够让队友支持你，甚至蒙蔽好人，为自己赢得更多信任。这种经历培养了在游戏中必备的自信心。2、逻辑思维的完善拥有良好的心态是一方面，而另一方面是需要强大的伪逻辑。悍跳训练时，你需要利用信息寻找漏洞，打破预言家和站在他一边的好人。同时，要处理盘对立面、共边的关系，抉择踩谁、拉谁的票等，这些都需要高度的逻辑思维。如果悍跳失败，虚心学习并改正逻辑问题是关键，这需要不断的实战练习。3、转变观念与心态调整不要认为成为狼就是见不得人，游戏中没有正义邪恶之分。将狼人看作正义的角色，在发言时展现出气势，心态是提升水平的关键。如果难以转变观念，可以通过寻找玩狼人的乐趣，如观看挂相。挑战自己找到神牌，指刀成功后的成就感，能够为你的心态调整提供积极的动力。4、学习挂相技巧通过悍跳过程，你可以学到观察挂相的技巧。不仅在狼人身上适用，好人同样能够通过观察挂相推断身份。掌握挂相技巧在面杀中非常厉害，且懂得的人寥寥无几。狼人杀游戏充满了学问和魅力。作为资深玩家，这些建议是通过实践积累出来的提升方法，更注重实际操作而非理论。相信这些建议能够对你在狼人杀游戏中的表现有所帮助。不断挑战自己，享受游戏的过程，你会在狼人杀的世界中不断进步。</Response>'

In [26]:


tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map,
    trust_remote_code=True
)

# 准备模型
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# # 基础知识微调训练
# print("开始基础知识微调训练...")
# basic_tokenized = basic_dataset.map(tokenize_function, batched=True, remove_columns=["text"])
# tokenized_dataset = dataset.map(
#     tokenize_function,
#     batched=True,
#     remove_columns=["text"]
# )


trainable params: 4,702,208 || all params: 1,548,416,512 || trainable%: 0.3037


In [32]:
# print(tokenized_dataset["train"])  # First 5 training samples

In [27]:
# 基础知识微调训练
print("开始基础知识微调训练...")
basic_tokenized = basic_dataset.map(tokenize_function, batched=True, remove_columns=["text"])

basic_training_args = TrainingArguments(
    output_dir=basic_model_path,
    num_train_epochs=3,  # 基础知识微调可以使用较少的epoch
    per_device_train_batch_size=4,     # -> next time to 4
    gradient_accumulation_steps=8,      # -> next time to 8   # 等效批量=32
    learning_rate=2e-4,  # 基础知识微调可以使用稍高的学习率
    fp16=True,
    eval_steps=10,
    logging_steps=1,
    eval_strategy="steps",
    save_strategy="steps",
    save_steps=50,
)

swanlab_callback = SwanLabCallback()

basic_trainer = Trainer(
    model=model,
    args=basic_training_args,
    train_dataset=basic_tokenized["train"],
    eval_dataset=basic_tokenized["test"],
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
    callbacks=[swanlab_callback]
)
basic_trainer.train()
print("基础知识微调完成!")

# 新增保存逻辑
# basic_model_path = "./lora_basic"  # 指定基础微调保存路径
model.save_pretrained(basic_model_path)  # 保存LoRA适配器参数[8](@ref)
tokenizer.save_pretrained(basic_model_path)

开始基础知识微调训练...


Map: 100%|██████████| 76/76 [00:00<00:00, 2020.21 examples/s]
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/site-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss
10,3.323800,2.825298
20,2.455500,2.652347
30,2.701800,2.573158
40,2.577800,2.529338
50,2.795700,2.499466
60,2.586000,2.485697


swanlab: Step 1 on key train/loss already exists, ignored.
swanlab: Step 1 on key train/grad_norm already exists, ignored.
swanlab: Step 1 on key train/learning_rate already exists, ignored.
swanlab: Step 1 on key train/epoch already exists, ignored.
swanlab: Step 1 on key train/global_step already exists, ignored.
swanlab: Step 1 on key train/loss already exists, ignored.
swanlab: Step 1 on key train/grad_norm already exists, ignored.
swanlab: Step 1 on key train/learning_rate already exists, ignored.
swanlab: Step 1 on key train/epoch already exists, ignored.
swanlab: Step 1 on key train/global_step already exists, ignored.
swanlab: Step 2 on key train/loss already exists, ignored.
swanlab: Step 2 on key train/grad_norm already exists, ignored.
swanlab: Step 2 on key train/learning_rate already exists, ignored.
swanlab: Step 2 on key train/epoch already exists, ignored.
swanlab: Step 2 on key train/global_step already exists, ignored.
swanlab: Step 2 on key train/loss already exists,

/usr/local/lib/python3.11/site-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


swanlab: Step 51 on key train/loss already exists, ignored.
swanlab: Step 51 on key train/grad_norm already exists, ignored.
swanlab: Step 51 on key train/learning_rate already exists, ignored.
swanlab: Step 51 on key train/epoch already exists, ignored.
swanlab: Step 51 on key train/global_step already exists, ignored.
swanlab: Step 52 on key train/loss already exists, ignored.
swanlab: Step 52 on key train/grad_norm already exists, ignored.
swanlab: Step 52 on key train/learning_rate already exists, ignored.
swanlab: Step 52 on key train/epoch already exists, ignored.
swanlab: Step 52 on key train/global_step already exists, ignored.
swanlab: Step 53 on key train/loss already exists, ignored.
swanlab: Step 53 on key train/grad_norm already exists, ignored.
swanlab: Step 53 on key train/learning_rate already exists, ignored.
swanlab: Step 53 on key train/epoch already exists, ignored.
swanlab: Step 53 on key train/global_step already exists, ignored.
swanlab: Step 54 on key train/loss

('./lora_basic_deepseek_r1/tokenizer_config.json',
 './lora_basic_deepseek_r1/special_tokens_map.json',
 './lora_basic_deepseek_r1/vocab.json',
 './lora_basic_deepseek_r1/merges.txt',
 './lora_basic_deepseek_r1/added_tokens.json',
 './lora_basic_deepseek_r1/tokenizer.json')

# SFT (12134 rows, train 10921, test 10%)

In [3]:
# 在SFT训练代码前重新初始化模型并加载基础微调结果：
# 重新初始化基础模型（重要！）
# 加载模型和分词器
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    # bnb_4bit_compute_dtype=torch.float16  # 强制使用 FP16 计算
)
tokenizer = AutoTokenizer.from_pretrained(model_name)  # 从基础微调保存路径加载
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    # gradient_checkpointing=True,#启用梯度检查点
    device_map=device_map,
    trust_remote_code=True
)
base_model = prepare_model_for_kbit_training(base_model)
# 加载第一阶段LoRA参数
model = get_peft_model(base_model, LoraConfig.from_pretrained(basic_model_path))
model.print_trainable_parameters()  # 验证参数加载

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


trainable params: 0 || all params: 1,548,416,512 || trainable%: 0.0000


In [4]:
# ==================== SFT微调部分 ====================
# SFT数据预处理函数
def format_sft_data(example):
    system_prompt = example["instruction"]
    user_input = example["prompt"]
    response = example["response"]
    # full_prompt = f"<Instruction>{system_prompt}</Instruction>\n<Prompt>{user_input}</Prompt>\n<Response>{response}</Response>"
    full_prompt = f"<Prompt>{user_input}</Prompt>\n<Response>{response}</Response>"
    # return {"text": full_prompt}
    return {"text": full_prompt.replace("\\n", "").replace("\n", "")}

# 加载SFT数据集
# sft_dataset = load_dataset("csv", data_files="train_zh.csv")["train"]
sft_dataset = load_dataset("csv", data_files="train_zh_part_1.csv")["train"]
# print(sft_dataset)
# sft_dataset = sft_dataset.select(range(2))  # 选择前5个样本
sft_dataset = sft_dataset.map(format_sft_data, remove_columns=["instruction", "prompt", "response", "meta"])
sft_dataset = sft_dataset.train_test_split(test_size=0.1)

In [5]:
sft_dataset['train']['text'][0]
# print(tokenizer.model_max_length)  # 查看模型支持的最大长度（如512）

'<Prompt>在本场游戏中，你目前已知以下信息：1. 角色设定：你是5号玩家。你的身份是：狼人。你需要其他狼人合作，每晚选择一个村民猎杀，你的目标是隐藏你的身份，并误导其他玩家，直至狼人获得游戏的胜利。2. 客观信息：- 游戏进程：目前游戏进行到第4轮。- 当前存活的玩家有：3号，4号，5号，6号，7号，8号，只能在以上玩家中选择进行杀害-狼人为：5,6,8号玩家。你是第一个行动的狼人，请选择你的杀害目标。- 投票情况：第1轮投票记录：1号玩家投给：8号玩家；2号玩家投给：9号玩家；3号玩家投给：9号玩家；4号玩家投给：9号玩家；5号玩家投给：9号玩家；6号玩家投给：9号玩家；8号玩家投给：9号玩家；9号玩家投给：8号玩家；结果：9号玩家被投票出局。第2轮投票记录：1号玩家投给：6号玩家；3号玩家投给：1号玩家；4号玩家投给：6号玩家；5号玩家投给：1号玩家；6号玩家投给：1号玩家；8号玩家投给：1号玩家；结果：1号玩家被投票出局。第3轮投票记录：3号玩家投给：8号玩家；4号玩家投给：8号玩家；5号玩家投给：3号玩家；6号玩家投给：3号玩家；7号玩家投给：8号玩家；8号玩家投给：3号玩家；结果：3，8号玩家平票，进入平票PK阶段。第3轮PK阶段投票记录：3号玩家投给：8号玩家；4号玩家投给：8号玩家；5号玩家投给：3号玩家；6号玩家投给：3号玩家；7号玩家投给：8号玩家；8号玩家投给：3号玩家；结果：3，8号玩家平票，进入平票PK阶段。3. 主观信息：第1轮总结：# 夜晚信息：  - 狼人，刀7号。- 平安夜，无人死亡。# 发言内容概括：**8号玩家**：- 跳预言家，9查杀。- 归票9号。**9号玩家**：- 声称自己是村民。- 归票8号。**1号玩家**：- 跳预言家，3号金水。- 晚上2号和7号选验。- 归票8号。**2号玩家**：- 认为1号和8号都有可能是预言家。- 归票9号。**3号玩家**：- 认为1号发言不像真预言家。- 归票9号。**4号玩家**：- 认为3号是好人。- 归票9号。**5号玩家**：- 赞同2号的想法。- 归票9号。**6号玩家**：- 认为9号是狼人。- 认为5号身份不好。**7号玩家**：- 分不清谁是真预言家。- 选择弃票。# 我的投票：- 我选择投票给9号玩家。第2轮总结：# 夜晚信息：  - 狼人，刀2号。- 2号死亡

# 爆显存, 需要将数据集 cut 一半试试, 主要 response 不能够被截断, 所以不能 truncation. 最长的输入序列是 7482.

In [6]:
# # 在基础知识训练结束后保存适配器参数
# basic_lora_weights = model.lora_A.weight.detach().clone()

# # 在SFT训练开始前加载对比
# assert torch.allclose(model.lora_A.weight, basic_lora_weights), "参数未继承！"
# from torch.cuda.amp import GradScaler
# scaler = GradScaler()  # 创建梯度缩放器

# SFT微调训练
def tokenize_function(examples):  # 新增参数
    # 假设 response 以 "<Response>" 开始
    response_start_token_id = tokenizer.encode("<Response>", add_special_tokens=False)[0]
    # print("Response token ID:", response_start_token_id)  # 应为有效数值
    tokenized = tokenizer(
        examples["text"],
        # padding="longest",  # 动态填充到批次中最长样本的长度
        padding="max_length",
        truncation='only_first',      # 禁用截断（需确保所有样本长度 ≤ max_length）
        max_length=3000,
        return_tensors="pt"
    )
    labels = tokenized["input_ids"].clone()
    for i in range(len(labels)):
        # 找到 <Response> 的起始位置
        start_pos = (tokenized["input_ids"][i] == response_start_token_id).nonzero(as_tuple=True)[0]
        if len(start_pos) > 0:
            labels[i, :start_pos.item()] = -100  # 忽略 instruction + prompt 的 loss
    tokenized["labels"] = labels
    
    # tokenized["labels"] = tokenized["input_ids"].clone()  # 直接复制输入作为标签
    return tokenized

print("开始SFT微调训练...")
sft_tokenized = sft_dataset.map(tokenize_function, batched=True, remove_columns=["text"])

开始SFT微调训练...


Map: 100%|██████████| 405/405 [00:00<00:00, 736.55 examples/s]


In [9]:
sft_training_args = TrainingArguments(
    output_dir=sft_model_path,  # 新保存路径
    num_train_epochs=2,  # 增加训练轮次
    per_device_train_batch_size=2,  # 增大批量大小
    gradient_accumulation_steps=2,  # 增加梯度累积步数
    learning_rate=2e-4,  # 调整学习率
    # fp16=True,
    bf16=True,
    max_grad_norm=1.0,
    fp16_full_eval=False,  # 禁用评估阶段的混合精度
    gradient_checkpointing=True,  # 启用梯度检查点优化显存[4]
    eval_steps=10,  # 降低评估步数以适配小数据量
    logging_steps=1,      # 每个训练步骤记录日志
    eval_strategy="steps",
    save_strategy="steps",
    save_steps=100,
    # logging_dir="./logs",  # 新增TensorBoard日志目录
    # report_to=["tensorboard"],  # 启用TensorBoard报告
    # load_best_model_at_end=True  # 自动加载最佳模型
)

swanlab_callback = SwanLabCallback()

# 创建Trainer
sft_trainer = Trainer(
    model=model,
    args=sft_training_args,
    train_dataset=sft_tokenized["train"],
    eval_dataset=sft_tokenized["test"],
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
    callbacks=[swanlab_callback]
)
# torch.autograd.set_detect_anomaly(True)  # 开启梯度异常检测
sft_trainer.train()

# 保存最终模型
# model.save_pretrained("./lora_final")
tokenizer.save_pretrained(sft_model_path)
print(f"模型已保存至 {sft_model_path}")



No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss,Validation Loss


swanlab: Step 1 on key train/loss already exists, ignored.
swanlab: Step 1 on key train/grad_norm already exists, ignored.
swanlab: Step 1 on key train/learning_rate already exists, ignored.
swanlab: Step 1 on key train/epoch already exists, ignored.
swanlab: Step 1 on key train/global_step already exists, ignored.
swanlab: Step 1 on key train/loss already exists, ignored.
swanlab: Step 1 on key train/grad_norm already exists, ignored.
swanlab: Step 1 on key train/learning_rate already exists, ignored.
swanlab: Step 1 on key train/epoch already exists, ignored.
swanlab: Step 1 on key train/global_step already exists, ignored.
swanlab: Step 2 on key train/loss already exists, ignored.
swanlab: Step 2 on key train/grad_norm already exists, ignored.
swanlab: Step 2 on key train/learning_rate already exists, ignored.
swanlab: Step 2 on key train/epoch already exists, ignored.
swanlab: Step 2 on key train/global_step already exists, ignored.
swanlab: Step 2 on key train/loss already exists,

OutOfMemoryError: CUDA out of memory. Tried to allocate 13.59 GiB. GPU 0 has a total capacity of 23.69 GiB of which 12.63 GiB is free. Process 163820 has 11.06 GiB memory in use. Of the allocated memory 9.76 GiB is allocated by PyTorch, and 1.02 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [26]:
# for log in trainer.state.log_history:
#     print(log)
# # 提取训练日志（包含 loss 或 train_loss 和 step）
# train_logs = []
# for log in trainer.state.log_history:
#     if "step" in log:
#         if "loss" in log:
#             log["train_loss"] = log["loss"]  # 统一字段名为 train_loss
#         train_logs.append(log)

# # 转换为 DataFrame
# train_df = pd.DataFrame(train_logs)[["step", "train_loss"]]
# # 提取评估日志（包含 eval_loss 和 step）
# eval_logs = [log for log in trainer.state.log_history if "eval_loss" in log and "step" in log]
# eval_df = pd.DataFrame(eval_logs)[["step", "eval_loss"]] if eval_logs else pd.DataFrame()
# # 合并训练和评估日志
# merged = pd.merge(train_df, eval_df, on="step", how="outer").sort_values("step")
# merged.ffill(inplace=True)  # 使用 ffill() 替代 fillna(method="ffill")
# plt.figure(figsize=(12, 6))
# plt.plot(merged["step"], merged["train_loss"], 'b-', label='Training Loss')
# if not eval_df.empty:
#     plt.plot(merged["step"], merged["eval_loss"], 'r--', label='Validation Loss')
# plt.title("Training Progress Analysis")
# plt.xlabel("Training Steps")
# plt.ylabel("Loss Value")
# plt.legend()
# plt.grid(True)
# plt.show()

# 生成任务评估（BLEU/ROUGE/METEOR）

In [28]:
import torch
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from nltk.translate.meteor_score import meteor_score
import jieba  # 中文分词支持

# 加载原始模型和微调模型
base_model = AutoModelForCausalLM.from_pretrained(model_name)
# finetuned_model = PeftModel.from_pretrained(base_model, output_dir)
finetuned_model = PeftModel.from_pretrained(base_model, basic_model_path)

# 加载分词器
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 加载测试数据
# df = pd.read_csv("train_zh.csv").tail(2)
df = pd.read_csv("game_strategy_and_term.csv").head(2)
test_dataset = Dataset.from_pandas(df)


In [29]:
import logging
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import jieba
from torch.nn import CrossEntropyLoss

# 关闭jieba调试日志
logging.getLogger("jieba").setLevel(logging.WARNING)

# 中文分词工具（改用搜索引擎模式提升召回率）
def chinese_tokenize(text):
    return list(jieba.cut_for_search(text))  # 使用搜索引擎模式[[2]]

# # 生成预测文本（增加生成长度）
# def generate_response(model, instruction, prompt):
#     input_text = f"[INST] <<SYS>>\n{instruction}\n<</SYS>>\n\n{prompt} [/INST]"
#     inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
#     outputs = model.generate(**inputs, max_new_tokens=100)  # 增加生成长度[[1]]
#     return tokenizer.decode(outputs[0], skip_special_tokens=True)

# 生成预测文本（增加生成长度）
def generate_response(model, prompt):
    input_text = f"{prompt}"
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=400)  # 增加生成长度
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# 显式设置pad_token_id（避免警告）
tokenizer.pad_token_id = tokenizer.eos_token_id

# 计算BLEU/ROUGE（增加平滑函数和ROUGE-L）
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=False)
smoother = SmoothingFunction()

def calculate_perplexity(model, tokenizer, prompt, response):
    """
    计算模型对给定prompt和response的困惑度
    """
    full_text = prompt + " " + response  # 拼接输入与响应
    inputs = tokenizer(full_text, return_tensors="pt").to(model.device)
    
    # 构造标签：仅计算response部分的loss
    prompt_len = len(tokenizer(prompt, add_special_tokens=False)['input_ids'])
    labels = inputs['input_ids'].clone()
    labels[:, :prompt_len] = -100  # 忽略prompt部分的loss计算
    
    with torch.no_grad():
        outputs = model(**inputs, labels=labels)
        loss = outputs.loss
    
    return torch.exp(loss).item()  # 返回困惑度

for i, example in enumerate(test_dataset):
    # instruction = example["instruction"]#######################################for sft prompt
    # print(f"指令: {instruction}")
    prompt = example["prompt"]
    # print(f"输入: {prompt}")
    reference = example["response"]
    # print(f"参考答案: {reference}")
    
    # 生成预测
    # base_output = generate_response(base_model, instruction, prompt)#######################################for sft prompt
    # ft_output = generate_response(finetuned_model, instruction, prompt)#######################################for sft prompt
    base_output = generate_response(base_model, prompt)
    ft_output = generate_response(finetuned_model, prompt)
    # 新增：计算困惑度
    base_ppl = calculate_perplexity(base_model, tokenizer, prompt, reference)
    ft_ppl = calculate_perplexity(finetuned_model, tokenizer, prompt, reference)
    
    # 分词处理
    ref_tokens = chinese_tokenize(reference)
    base_tokens = chinese_tokenize(base_output)
    ft_tokens = chinese_tokenize(ft_output)
    
    # BLEU优化：使用BLEU-2+平滑函数
    bleu_base = sentence_bleu(
        [ref_tokens], base_tokens, 
        weights=(0.5, 0.5),  # 使用BLEU-2
        smoothing_function=smoother.method1  # 添加平滑
    )
    bleu_ft = sentence_bleu(
        [ref_tokens], ft_tokens,
        weights=(0.5, 0.5),
        smoothing_function=smoother.method1
    )
    
    # ROUGE优化：使用分词后的文本并增加ROUGE-L
    rouge_base = scorer.score(
        " ".join(ref_tokens), 
        " ".join(base_tokens)
    )
    rouge_ft = scorer.score(
        " ".join(ref_tokens), 
        " ".join(ft_tokens)
    )
    # 新增打印内容：对比生成结果与参考答案
    print(f"\n{'='*20} 样本{i+1} 原始内容 {'='*20}")
    # print(f"【指令】: {instruction}")
    # print(f"【输入】: {prompt}")
    print(f"【参考回答】: {reference}")
    print(f"\n【基础模型生成】: {base_output}")
    print(f"【微调模型生成】: {ft_output}")
    
    # 评估指标打印（保持原有格式）
    print(f"\n{'='*20} 样本{i+1} 评估结果 {'='*20}")
    print(f"BLEU-2 - 原始: {bleu_base:.4f}, 微调: {bleu_ft:.4f}")
    print(f"ROUGE-2 F1 - 原始: {rouge_base['rouge2'].fmeasure:.4f}, 微调: {rouge_ft['rouge2'].fmeasure:.4f}")
    print(f"ROUGE-L F1 - 原始: {rouge_base['rougeL'].fmeasure:.4f}, 微调: {rouge_ft['rougeL'].fmeasure:.4f}")
    print("-"*50 + "\n")
    
    # 打印困惑度结果
    print(f"Perplexity - 原始: {base_ppl:.2f}, 微调: {ft_ppl:.2f}")
    print("-"*50 + "\n")
    
    # print(f"样本{i+1}评估结果:")
    # print(f"BLEU-2 - 原始: {bleu_base:.4f}, 微调: {bleu_ft:.4f}")
    # print(f"ROUGE-2 F1 - 原始: {rouge_base['rouge2'].fmeasure:.4f}, 微调: {rouge_ft['rouge2'].fmeasure:.4f}")
    # print(f"ROUGE-L F1 - 原始: {rouge_base['rougeL'].fmeasure:.4f}, 微调: {rouge_ft['rougeL'].fmeasure:.4f}")
    # print("-"*50)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



==================== 样本1 原始内容 ====================
【参考回答】: 在狼人杀这款推理游戏中，猎人作为一个关键角色，其玩法策略极为重要。猎人具有独特的技能——在死亡时可以带走一名玩家，这使得猎人在游戏中具有不可忽视的影响力。本文将详细介绍猎人的玩法技巧，帮助新手玩家更好地掌握这一角色。
首先，猎人的起跳时机至关重要。由于猎人的身份较为特殊，有时会有狼人选择悍跳猎人。因此，猎人需要在合适的时机表明自己的身份，以确保好人阵营的优势。在起跳之前，猎人需要充分观察场上的局势，确保自己的身份能够得到大家的认可。
其次，猎人的枪法也是玩法中的一大关键。由于猎人在死亡时可以带走一名玩家，但无法得知该玩家的真实身份，因此开枪的时机和选择至关重要。在没有十足把握的情况下，猎人应该谨慎开枪，避免误杀好人，特别是预言家和女巫等重要角色。在前期，猎人可以尽量保持低调，通过观察和推理积累信息，以便在关键时刻做出正确的决策。
在游戏进行到中段时，猎人需要注意女巫的动向。当女巫跳出来表态并表明自己救了谁时，猎人应该勇敢地站出来表态，以避免被女巫误杀。这是因为女巫在投毒时，猎人无法触发技能，因此及时表明身份是保护自己的重要手段。
此外，猎人在游戏中还需要保持冷静和理智。在面对狼人的挑衅和质疑时，猎人需要坚定自己的立场，用逻辑和证据来回应。同时，猎人也需要与队友保持良好的沟通，共同分析局势，制定合适的策略。
总之，猎人在狼人杀游戏中具有重要的作用。通过掌握起跳时机、谨慎开枪、注意女巫动向以及保持冷静理智等技巧，新手玩家可以更好地扮演猎人这一角色，为好人阵营的胜利贡献自己的力量。希望本文的攻略能够帮助你在狼人杀游戏中取得更好的成绩！

【基础模型生成】: 狼人杀中，如何扮演好猎人角色并为好人阵营取得胜利？ 在狼人杀游戏中，猎人是一个非常重要的角色，他的主要任务是通过毒杀狼人来帮助好人阵营获胜。然而，猎人并不是一个简单的角色，需要玩家具备一定的策略和技巧。本文将为大家介绍如何扮演好猎人角色并为好人阵营取得胜利。
1. 了解猎人的技能和特点
猎人是狼人杀游戏中最强大的角色之一，他的技能是毒杀狼人。然而，猎人也有自己的特点，比如他需要在夜晚进行毒杀，而且毒杀的狼人必须是自己观察到的。此外，猎人还需要注意自己的行动，避免被狼人发现并被狼人攻击。
2. 选择合适

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



==================== 样本2 原始内容 ====================
【参考回答】: 狼人杀作为一款心理推理游戏，不仅考验玩家的逻辑能力和洞察力，还涉及到发言的重要性。然而，随着游戏的深入，一些玩家可能会尝试篡改发言，给游戏增添了一些挑战和争议。在面对这种情况时，我们应该如何应对呢？下面就让我们来探讨一下。
首先，我们要认识到篡改发言在狼人杀游戏中并不罕见。由于每个人对他人发言的理解有所偏差，篡改发言往往是人们根据自己的价值观进行选择性截取或修改后的产物。因此，不能简单地将篡改发言视为狼的行为，而是要理解其背后的动机和原因。
其次，发言内容并不是唯一的判断依据。在狼人杀游戏中，玩家可以通过观察卦相、表情、状态等方式来推断其他玩家的身份。因此，即使发言被篡改，仍然有其他方式可以辅助确认玩家的真实身份。我们应该摆脱对发言内容的过度依赖，而是综合考虑多种因素进行判断。
针对篡改发言行为，我们可以采取一些策略来有效地应对。首先，要尽量保持冷静，不要被篡改发言所影响，而是要坚持自己的判断和观点。其次，可以通过提出问题或向其他玩家求证来验证发言的真实性，从而避免被误导。此外，如果发现有玩家频繁篡改发言，可以在游戏规则中加入相应的限制或惩罚，以维护游戏的公平性和趣味性。
最后，我们要记住，狼人杀是一款旨在娱乐和交流的游戏，而不是严肃的逻辑竞技。在游戏中，我们应该注重的是玩家之间的互动和合作，而不是纠缠于发言的细节和篡改行为。只有在轻松愉快的氛围中，才能真正体验到狼人杀游戏的乐趣。
综上所述，面对狼人杀中的发言篡改行为，我们应该以宽容和理解的态度去应对，同时采取相应的策略来应对挑战。重要的是要保持冷静，不受干扰，从而更好地享受游戏带来的乐趣。

【基础模型生成】: 狼人杀游戏中如何应对玩家篡改发言的行为？ 在狼人杀游戏中，玩家篡改发言是一种常见的行为，它可能源于各种原因，如紧张、情绪激动或试图掩盖自己的身份。面对这种情况，作为玩家，应该采取以下策略来应对：

1. **保持冷静**：首先，不要被其他玩家的发言所影响，保持自己的冷静。情绪激动可能会导致你做出错误的判断。

2. **观察其他玩家的行为**：注意其他玩家的发言和行为，看看他们是否真的有篡改发言的动机。有时候，玩家可能会因为紧张而误报身份。

3. **分析发言内容**：仔细分析其他玩家